In [3]:
!pip install -U langchain-huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import ollama

DOCUMENT_PATHS = [
    r'/home/masih/rag_data/Hamrah.txt',
    r'/home/masih/rag_data/vape.txt',
    r'/home/masih/rag_data/Shah.txt',
    r'/home/masih/rag_data/Khalife.txt',
    r'/home/masih/rag_data/carbon.txt',
    r'/home/masih/rag_data/takapoo.txt'
]

# Load all documents
def load_documents(file_paths):
    texts = []
    for file_path in file_paths:
        loader = TextLoader(file_path)
        documents = loader.load()
        texts.extend(documents)
    return texts

texts = load_documents(DOCUMENT_PATHS)

# Init embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2") 

# Create a vector store 4 semantic search
index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=embeddings
).from_documents(texts)

# Init Llama3.2 locally
llm = Ollama(model="llama3.2")

# Create a chat interface
template = PromptTemplate.from_template(
    "You are a helpful assistant with access to the following documents:\n\n{documents}\n\nPlease answer the question: {question}\n"
)

def chat_with_data():
    # Create an LLM chain for responses
    chain = LLMChain(llm=llm, prompt=template)
    
    # Simple chat loop
    while True:
        question = input("You: ")
        response = chain.run(question=question, documents=texts)
        print("Assistant:", response)

# Start the chat
chat_with_data()

/tmp/ipykernel_98350/1857655220.py:42: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")
/tmp/ipykernel_98350/1857655220.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=template)


You:  what's the gist of my data


/tmp/ipykernel_98350/1857655220.py:56: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(question=question, documents=texts)


KeyboardInterrupt: 